# Livro

In [21]:
import pandas as pd
import nltk
import xmltodict
from dicttoxml import dicttoxml
from xml.dom.minidom import parseString

In [22]:
%run ../funcoes.py

Loading functions...


In [26]:
# carregando tipos
domain = "livro"
tiposAll = pd.read_csv('../data/tipologia/categorias_iacs.csv', sep=',', quoting=0, keep_default_na=False)
tipos = tiposAll.loc[tiposAll['domain'] == domain]

In [3]:
dfTest = pd.read_csv('../07_Keras/data/domain_livro_IOB_test.csv', sep='\t', quoting=3, keep_default_na=False)
dfTrain = pd.read_csv('../07_Keras/data/domain_livro_IOB_train.csv', sep='\t', quoting=3, keep_default_na=False)

In [4]:
f = open("../07_Keras/data/aspectsTest.xml","r")
xml = f.read()
aspectsTest = xmltodict.parse(xml, process_namespaces=True)

In [5]:
def result2aspecList(y_pred, X_test):
    aspectsList = []
    aspect = []
    stopwords = nltk.corpus.stopwords.words('portuguese')
    for sent, sent_pred in zip(X_test, y_pred):
        #print(len(sent),len(sent_pred))
        for token, tag in zip(sent, sent_pred):
            if tag != 'O':
                #print(tag, token)
                #print(tag, token['word.lower()'])
                if tag == 'B-asp' and aspect:
                    auxAspect = " ".join(aspect).lower()
                    if (auxAspect not in stopwords and len(auxAspect) > 1):
                        aspectsList.append(auxAspect)
                    aspect = []
                if token not in [',',':']:
                    aspect.append(token.lower())

    aspectsList = list(set(aspectsList))
    aspectsList.sort()    
    return aspectsList

In [6]:
tag_scheme = [
'B-asp',
'I-asp'
]

In [7]:
contSents = 0
for count, row in dfTrain.iterrows():
    if (row[0] != "" and count > 0):
        contSents = contSents + 1
contSents = contSents + 1

In [8]:
contTrain = round(contSents * 0.9)
print(contTrain)

218


In [9]:
training = {'sentences': [], 'tags' : []}
validation = {'sentences': [], 'tags' :[]}
sent = []
tags = []
contSents = 0
for count, row in dfTrain.iterrows():
    if (row[0] != "" and count > 0):
        if contSents < contTrain:
            training['sentences'].append(sent)
            training['tags'].append(tags)
        else:
            validation['sentences'].append(sent)
            validation['tags'].append(tags)

        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
        
validation['sentences'].append(sent)
validation['tags'].append(tags)

In [10]:
len(training['sentences'])

218

In [11]:
len(validation['sentences'])

24

In [12]:
test = {'sentences': [], 'tags' : []}
sent = []
tags = []
contSents = 0
for count, row in dfTest.iterrows():
    if (row[0] != "" and count > 0):
        test['sentences'].append(sent)
        test['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
test['sentences'].append(sent)
test['tags'].append(tags)

In [13]:
len(test['sentences'])

88

In [14]:
transformer = 'bert-base-multilingual-uncased'

In [15]:
# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
    'epochs' : 40,
    'warmup_steps' : 500,                                                   
    'train_batch_size': 13,
    'learning_rate': 0.0001
}

In [16]:
from NERDA.models import NERDA
model = NERDA(
    dataset_training = training,
    dataset_validation = validation,
    tag_scheme = tag_scheme, 
    tag_outside = 'O',
    transformer = transformer,
    dropout = dropout,
    hyperparameters = training_hyperparameters
)

2021-12-21 09:02:20.812086: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-21 09:02:20.812123: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Device automatically set to: cpu


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
model.train()


 Epoch 1 / 40


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:07<00:00,  2.41s/it]


Train Loss = 0.9067689846543705 Valid Loss = 0.4194265405337016

 Epoch 2 / 40


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.26s/it]


Train Loss = 0.31067371017792644 Valid Loss = 0.22537411749362946

 Epoch 3 / 40


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:07<00:00,  2.60s/it]


Train Loss = 0.0005916353821864023 Valid Loss = 0.18206404938246123

 Epoch 40 / 40


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.83s/it]

Train Loss = 0.0012985296746003715 Valid Loss = 0.18222476456624767


'Model trained successfully'

In [18]:
model.evaluate_performance(test)

/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #70 length 149 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 5 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 5 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'l

,Level,F1-Score,Precision,Recall
0,B-asp,0.771084,0.800000,0.744186
1,I-asp,0.142857,0.333333,0.090909
0,AVG_MICRO,0.722222,NaN,NaN
0,AVG_MICRO,0.456971,NaN,NaN


In [19]:
y_pred = model.predict(test['sentences'])

/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #70 length 149 exceeds max_len 128 and has been truncated
  warnings.warn(msg)


In [20]:
aspects = result2aspecList(y_pred, test['sentences'])
res = calculaMetricas(aspects, aspectsTest['datasets']['livro'], lemma=False, output=False)
res

{'totalAspects': 87,
 'pred': 32,
 'tp': 58,
 'fp': 10,
 'fn': 29,
 'precisao': 0.8529411764705882,
 'precisaoLaplace': 0.8428571428571429,
 'revocacao': 0.6666666666666666,
 'medidaf': 0.7483870967741936,
 'totalExplicitos': 82,
 'explicitosOk': 22,
 'pExplicitos': 0.2682926829268293,
 'totalImplicitos': 5,
 'implicitosOk': 0,
 'pImplicitos': 0.0}

In [29]:
aspects = result2aspecList(y_pred, test['sentences'])
res = calculaMetricas(
    aspects, 
    aspectsTest['datasets'][domain], 
    tipos,
    lemma=False, 
    output=False
)

xml = dicttoxml(res, attr_type=False, custom_root='resultados')
with open('../resultados/bert_multi_indomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())    
res

{'totalAspects': 87,
 'pred': 32,
 'tp': 58,
 'fp': 10,
 'fn': 29,
 'precisao': 0.8529411764705882,
 'precisaoLaplace': 0.8428571428571429,
 'revocacao': 0.6666666666666666,
 'medidaf': 0.7483870967741936,
 'totalExplicitos': 82,
 'explicitosOk': 22,
 'pExplicitos': 0.2682926829268293,
 'explicitsList': ['história',
  'leitura',
  'estilo',
  'fim',
  'palavra',
  'trama',
  'historia',
  'final',
  'romance',
  'protagonista',
  'narrador',
  'personagem',
  'obra',
  'início',
  'livro',
  'crítica',
  'personagens',
  'situaçoês',
  'começo',
  'escritor',
  'leituras',
  'narrativa'],
 'totalImplicitos': 5,
 'implicitosOk': 0,
 'pImplicitos': 0.0,
 'implicitsList': [],
 'tipos': {}}

## Bert Portuguese: Bertimbau

In [30]:
transformer = 'neuralmind/bert-base-portuguese-cased'

# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
    'epochs' : 40,
    'warmup_steps' : 500,                                                   
    'train_batch_size': 13,
    'learning_rate': 0.0001
}

from NERDA.models import NERDA
model2 = NERDA(
    dataset_training = training,
    dataset_validation = validation,
    tag_scheme = tag_scheme, 
    tag_outside = 'O',
    transformer = transformer,
    dropout = dropout,
    hyperparameters = training_hyperparameters
)

Device automatically set to: cpu


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
model2.train()


 Epoch 1 / 40


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:12<00:00,  4.10s/it]


Train Loss = 1.1136757941807018 Valid Loss = 0.6132064859072367

 Epoch 2 / 40


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:12<00:00,  4.32s/it]


Train Loss = 0.32056593982612386 Valid Loss = 0.223128875096639

 Epoch 3 / 40


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:12<00:00,  4.07s/it]


Train Loss = 0.0002293812152963844 Valid Loss = 0.12008592238028844

 Epoch 40 / 40


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:12<00:00,  4.29s/it]


Train Loss = 0.00023127117267850896 Valid Loss = 0.12012695583204429


'Model trained successfully'

In [32]:
model2.evaluate_performance(test)

/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #70 length 161 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 5 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 5 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'l

,Level,F1-Score,Precision,Recall
0,B-asp,0.795181,0.814815,0.776471
1,I-asp,0.142857,0.333333,0.090909
0,AVG_MICRO,0.744444,NaN,NaN
0,AVG_MICRO,0.469019,NaN,NaN


In [33]:
y_pred = model2.predict(test['sentences'])

/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #70 length 161 exceeds max_len 128 and has been truncated
  warnings.warn(msg)


In [34]:
aspects = result2aspecList(y_pred, test['sentences'])
res = calculaMetricas(
    aspects, 
    aspectsTest['datasets'][domain], 
    tipos,
    lemma=False, 
    output=False
)
xml = dicttoxml(res, attr_type=False, custom_root='resultados')
with open('../resultados/bert_bertimbau_indomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())    
res

{'totalAspects': 87,
 'pred': 33,
 'tp': 59,
 'fp': 10,
 'fn': 28,
 'precisao': 0.855072463768116,
 'precisaoLaplace': 0.8450704225352113,
 'revocacao': 0.6781609195402298,
 'medidaf': 0.7564102564102564,
 'totalExplicitos': 82,
 'explicitosOk': 23,
 'pExplicitos': 0.2804878048780488,
 'explicitsList': ['história',
  'leitura',
  'estilo',
  'fim',
  'palavra',
  'trama',
  'historia',
  'final',
  'romance',
  'protagonista',
  'narrador',
  'personagem',
  'obra',
  'início',
  'livro',
  'crítica',
  'personagens',
  'situação',
  'situaçoês',
  'começo',
  'escritor',
  'leituras',
  'narrativa'],
 'totalImplicitos': 5,
 'implicitosOk': 0,
 'pImplicitos': 0.0,
 'implicitsList': [],
 'tipos': {}}